# RFM 集計ノード

In [1]:
import pandas as pd
import datetime

In [2]:
df_tran = pd.read_csv("../data/sampletranDEPT4en2019.csv", parse_dates=['SDATE'])
df_tran.head()


,CUSTID,SDATE,PRODUCTID,L_CLASS,M_CLASS,AMOUNT,U_PRICE,SUBTOTAL
0,100001,2019-02-26,9900200,UNDERWEAR,UNDERWEAR06,2,700,1400
1,100001,2019-02-26,9902142,FOOD,FOOD12,2,840,1680
2,100001,2019-07-02,9937845,BAG,BAG04,5,348,1740
3,100001,2019-07-02,9902517,FOOD,FOOD12,5,550,2750
4,100001,2019-07-09,9903878,BAG,BAG04,7,444,3108


In [3]:
# 購買履歴データ内の最新日時を取得し、Recencyの基準日とする
NOW=df_tran["SDATE"].max()

# 指定日以降のトランザクションデータを集計の対象とする
df_tran=df_tran[df_tran["SDATE"]>="2019-09-01"]

# 指定の値以下のレコードを破棄
df_tran=df_tran[df_tran["SUBTOTAL"]>2000]

# Recency,Frequency,Monetary,R2,R3を集計する
ID="CUSTID"
df_rfm = (
    df_tran.groupby(ID)
    .agg(
        {
            "SDATE": [
                lambda date: (NOW - date.max()).days,
                lambda date: len(date),
                lambda date: (NOW - sorted(date)[-2]).days
                if len(date) > 1
                else pd.NA,
                lambda date: (NOW - sorted(date)[-3]).days
                if len(date) > 2
                else pd.NA,
            ],
            "SUBTOTAL": sum,
        }
    )
    .reset_index()
)
# カラム名の変更と順番の並び替え
df_rfm.columns=[ID,"Recency","Frequency","R2","R3","Monetary"]
df_rfm=df_rfm.reindex(columns=[ID,"Recency","Frequency","Monetary","R2","R3"])



In [4]:
df_rfm

,CUSTID,Recency,Frequency,Monetary,R2,R3
0,100001,9,10,57397,59,59
1,100004,20,8,34711,29,29
2,100005,77,2,10700,108,<NA>
3,100006,41,2,6700,104,<NA>
4,100008,3,3,8789,3,120
...,...,...,...,...,...,...
2788,105967,2,16,84595,64,65
2789,105970,68,8,62506,79,85
2790,105972,34,8,28978,65,83
2791,105974,19,21,60742,19,22
